## SENG 474 Data Mining Project

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import sqrt

data = pd.read_csv('google-play-store-apps/googleplaystore.csv')
#wait like 30s for this to finish
# to_drop = ['Reviews',
#           'Size',
#           'Genres',
#           'Last Updated',
#           'Current Ver',
#           'Android Ver']
# data.drop(to_drop, inplace=True, axis=1)
data.head()


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [2]:
# installs = r'^(\d*)'
# extr = data["Installs"].str.replace(",", '')
# extr = extr.str.extract(installs)
# extr.head()

# Fixing Price
data = data.where(data['Price'] != "Everyone")
data["Price"] = data["Price"].str.replace("$", '')
data["Price"] = pd.to_numeric(data["Price"])
# data["Price"] = data.apply(lambda x: pd.to_numeric(x), axis=0)

# Fixing Installs
data["Installs"] = data["Installs"].str.replace(",", '')
data["Installs"] = data["Installs"].str.replace("+", '')
data["Installs"] = pd.to_numeric(data["Installs"])
# data["Installs"] = extr.apply(lambda x: pd.to_numeric(x), axis=0)
# data["Reviews"] = extr.apply(lambda x: pd.to_numeric(x), axis=0)

data["Reviews"] = pd.to_numeric(data["Reviews"])
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,19M,10000.0,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14M,500000.0,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510.0,8.7M,5000000.0,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644.0,25M,50000000.0,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967.0,2.8M,100000.0,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:

C = data['Rating'].mean()
print(C)

# Calculate the minimum number of votes required to be in the chart, m
m = data['Reviews'].quantile(0.90)
print(m)

# Filter out all qualified apps into a new DataFrame
q_apps = data.copy().loc[data['Reviews'] >= m]
q_apps.shape

# Function that computes the weighted rating of each app
def weighted_rating(x, m=m, C=C):
    v = x['Reviews']
    R = x['Rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_apps['score'] = q_apps.apply(weighted_rating, axis=1)

q_apps = q_apps.sort_values('score', ascending=False)

#Print the top 5 apps
q_apps.head(5)


# #Import TfIdfVectorizer from scikit-learn
# from sklearn.feature_extraction.text import TfidfVectorizer

# #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
# tfidf = TfidfVectorizer(stop_words='english')

# #Replace NaN with an empty string
# data['Category'] = data['Category'].fillna('')

# #Construct the required TF-IDF matrix by fitting and transforming the data
# tfidf_matrix = tfidf.fit_transform(data['Category'])

# #Output the shape of tfidf_matrix
# tfidf_matrix.shape


# # Import linear_kernel
# from sklearn.metrics.pairwise import linear_kernel

# # Compute the cosine similarity matrix
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# # Construct a reverse map of indices and app titles
# indices = pd.Series(data.index, index=data['App']).drop_duplicates()


# # Import CountVectorizer and create the count matrix
# from sklearn.feature_extraction.text import CountVectorizer

# count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(data['Category'])
# # Compute the Cosine Similarity matrix based on the count_matrix
# from sklearn.metrics.pairwise import cosine_similarity

# cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
# # Reset index of your main DataFrame and construct reverse mapping as before
# data = data.reset_index()
# indices = pd.Series(data.index, index=data['App'])




# def get_recommendations(title, cosine_sim=cosine_sim):
#     # Get the index of the app that matches the name
#     idx = indices[title]

#     # Get the pairwsie similarity scores of all apps with that app
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     # Sort the apps based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[0], reverse=True)

#     # Get the scores of the 10 most similar apps
#     sim_scores = sim_scores[1:11]

#     # Get the app indices
#     app_indices = [i[0] for i in sim_scores]

#     # Return the top 10 most similar app
#     return data['App'].iloc[app_indices]

# print(get_recommendations('Paint Space AR', cosine_sim2))

4.191757420456978
464993.1000000003


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,score
1360,Period Tracker - Period Calendar Ovulation Tra...,HEALTH_AND_FITNESS,4.8,4559407.0,Varies with device,100000000.0,Free,0.0,Everyone,Health & Fitness,"August 1, 2018",Varies with device,Varies with device,4.743709
4038,"DU Recorder – Screen Recorder, Video Editor, Live",VIDEO_PLAYERS,4.8,2588730.0,9.7M,50000000.0,Free,0.0,Everyone,Video Players & Editors,"July 30, 2018",1.7.1.3,5.0 and up,4.707382
4005,Clean Master- Space Cleaner & Antivirus,TOOLS,4.7,42916526.0,Varies with device,500000000.0,Free,0.0,Everyone,Tools,"August 3, 2018",Varies with device,Varies with device,4.694552
7536,"Security Master - Antivirus, VPN, AppLock, Boo...",TOOLS,4.7,24900999.0,Varies with device,500000000.0,Free,0.0,Everyone,Tools,"August 4, 2018",4.6.6,Varies with device,4.690683
1763,Piano Tiles 2™,GAME,4.7,8118880.0,Varies with device,100000000.0,Free,0.0,Everyone,Arcade,"August 3, 2018",3.1.0.226,4.1 and up,4.672468


In [ ]:
def sim_distance_pearson_score(app_data,p1,p2):
#     print ("############## Calculating Pearson Score ##############")
#     print ("Comparing \"", app_data[p1]["App"], "\" with", app_data[p2]["App"], "\"")

    si= { "Rating", "Installs", "Price"}
    
    #  How to compare category?
    # ADD N + 1 when comparing with category
    n = 3
    
    
    # Add up all the preferences
    sum1=sum([app_data[p1][it] for it in numeric_comparing_keys])
    sum2=sum([app_data[p2][it] for it in numeric_comparing_keys])
    
    # Sum up the squares
    sum1Sq=sum([pow(app_data[p1][it],2) for it in numeric_comparing_keys])
    sum2Sq=sum([pow(app_data[p2][it],2) for it in numeric_comparing_keys])
    
    # Sum up the products
    pSum=sum([app_data[p1][it]*app_data[p2][it] for it in numeric_comparing_keys])
    
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den
    return r

# 0.9999999497436914
# 0.9999999393128385

print (sim_distance_pearson_score(app_data,0,1324))



In [ ]:
# Returns the best matches for given app from the dictionary
def topMatches(prefs,person,n=5,similarity=sim_distance_pearson_score):
    scores=[(similarity(prefs,person,other),prefs[other]["App"])
    for other in prefs if other!=person]
    
    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

print(topMatches(app_data, 296))

In [ ]:
data.shape
data.plot(y=2)

In [ ]:
def count_unique(col):
    unique, counts = np.unique(col, return_counts=True)
    return np.array(list(zip(unique, counts)), dtype=object)

X = data.values

def plot_bar(col):
    categories = count_unique(col)

    fig, ax = plt.subplots(figsize=(18,8))
    y_pos = np.arange(len(categories[:, 0]))
    plt.bar(y_pos, categories[:, 1])
    plt.xticks(y_pos, categories[:, 0])
    plt.xticks(rotation=75)
    
plot_bar(X[:, 1])
#categories